<a href="https://colab.research.google.com/github/mett29/DL-Competition/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!cp kaggle.json ~/.kaggle/

In [13]:
from getpass import getpass
import os

user = getpass('Github user')
password = getpass('Github password')
os.environ['GITHUB_AUTH'] = user + ':' + password

!git clone https://$GITHUB_AUTH@github.com/mett29/DL-Competition.git

Github user··········
Github password··········
Cloning into 'DL-Competition'...
remote: Enumerating objects: 2087, done.
remote: Total 2087 (delta 0), reused 0 (delta 0), pack-reused 2087
Receiving objects: 100% (2087/2087), 90.16 MiB | 10.96 MiB/s, done.
Resolving deltas: 100% (5/5), done.
